# Inserting Rows with Core

Ketika menggunakan Core, SQL INSERT statement dihasilkan menggunakan fungsi <b>insert()</b>. Fungsi ini menghasilkan sebuah instance baru dari Insert yang merepresentasikan sebuah INSERT statement pada SQL yang menambahkan data baru ke dalam tabel.

In [1]:
from sqlalchemy import create_engine, MetaData

engine = create_engine("sqlite+pysqlite:///example.db", future=True)
metadata = MetaData(bind=engine)
metadata.reflect()

users_table = metadata.tables["users"]
email_table = metadata.tables["email"]

In [2]:
from sqlalchemy import insert
ins_stmt = insert(users_table).values(name="spongebob", fullname="spongebob squarepants")

In [3]:
print(ins_stmt)

INSERT INTO users (name, fullname) VALUES (?, ?)


In [5]:
ins_stmt.compile().params

{'name': 'spongebob', 'fullname': 'spongebob squarepants'}

## Executing the Statement

In [6]:
with engine.connect() as conn:
    result = conn.execute(ins_stmt)
    conn.commit()

In [8]:
result.inserted_primary_key

(1,)

## INSERT usually generates the “values” clause automatically

In [9]:
with engine.connect() as conn:
    result = conn.execute(
        insert(users_table),
        [
            {"name": "sandy", "fullname": "sandy cheeks"},
            {"name": "patrick", "fullname": "patrick star"}
        ]
    )
    
    conn.commit()

## INSERT…FROM SELECT

In [2]:
from sqlalchemy import select
sel_stmt = select(users_table.c.id, users_table.c.name + "@aol.com")

In [3]:
print(sel_stmt)

SELECT users.id, users.name || ? AS anon_1 
FROM users


In [5]:
from sqlalchemy import insert
ins_stmt = insert(email_table).from_select(
    ["user_id", "email_address"], sel_stmt
)

In [6]:
print(ins_stmt)

INSERT INTO email (user_id, email_address) SELECT users.id, users.name || ? AS anon_1 
FROM users


In [11]:
with engine.connect() as conn:
    result = conn.execute(ins_stmt)
    
    conn.commit()